# Analysis of Tesla PAB

This jupyter notebook presents the data along with the calculations for added transparency. Section 1 will read the data and create functions and plots, while [Section 2](#results) will show the data and plots.

To run all the cells, click **Kernel** > **Restart and Run All**.

Each cell can be run individually with **Shift + Enter**.

In [11]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

## 1 Preparation

### 1.1 Python Packages

Pandas will be used for manipulating the excel table data, while seaborn will help with displaying the plots.

In [1]:
import os
import pandas as pd
import seaborn as sns
sns.set(font_scale=2)
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
# https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916 

Next we read the excel sheets representing the front, and side data results from the MLCT program. We'll extract just the columns we need with the double brackets, then display the first 5 rows of the front dataframe (front_df).

In [2]:
# read excel sheets for AOA data
front_aoa_df = pd.read_excel(r'data/front_data.xlsx', index_col='completed')[['Frame', 'sample', 'Raw Score']]
side_aoa_df = pd.read_excel(r'data/side_data.xlsx', index_col='completed')[['Frame', 'sample', 'Raw Score']]

# Rename index column
front_aoa_df.index.name = 'Date'
side_aoa_df.index.name = 'Date'

# Remove first data point
front_aoa_df = front_aoa_df.iloc[7:]
side_aoa_df = side_aoa_df.iloc[7:]

# Show preview
front_aoa_df[front_aoa_df['Frame'] == 10].head()

,Frame,sample,Raw Score
Date,,,
2021-08-30 07:24:55,10,CS656175421068AAFC,1042.895535
2021-08-31 03:54:20,10,CS656175421069ABBO,1326.435597
2021-09-01 03:30:02,10,CS656175421069AAVF,1087.877107
2021-09-02 03:02:26,10,CS656175421069AAOQ,975.204843
2021-09-03 03:06:55,10,CS656175421069AAAD,1054.360063


In [3]:
side_aoa_df[side_aoa_df['Frame'] == 10].head()

,Frame,sample,Raw Score
Date,,,
2021-08-30 07:24:55,10,CS656175421068AAFC,1794.292271
2021-08-31 03:54:20,10,CS656175421069ABBO,1651.935818
2021-09-01 03:30:02,10,CS656175421069AAVF,1833.307798
2021-09-02 03:02:26,10,CS656175421069AAOQ,2063.317460
2021-09-03 03:06:55,10,CS656175421069AAAD,1528.661836


Next we'll create fake data for data from china (in the meantime before the data is gathered)

In [4]:
# Copy the first datasets
front_acs_df = front_aoa_df.copy()
side_acs_df = side_aoa_df.copy()

# Add random noise to scores
noise_range = 1500
front_acs_df['Raw Score'] = front_acs_df['Raw Score'] - noise_range/2 + np.random.randint(noise_range, size=len(front_acs_df))
side_acs_df['Raw Score'] =  side_acs_df['Raw Score'] - noise_range/2 + np.random.randint(noise_range, size=len(side_acs_df))

# Show preview
front_acs_df.head()

,Frame,sample,Raw Score
Date,,,
2021-08-27 03:44:59,13,CS656175421068AAMD,1021.293526
2021-08-27 03:44:59,40,CS656175421068AAMD,1385.565424
2021-08-27 03:44:59,16,CS656175421068AAMD,2001.247146
2021-08-27 03:44:59,25,CS656175421068AAMD,1091.966940
2021-08-27 03:44:59,19,CS656175421068AAMD,595.730894


### 1.2 $\bar{X}$ and MR Chart

We'll create a function that takes the dataframe, frame number (in milliseconds), and a time period to group data by. The function will provide an $\bar{X}$ and MR plot. (Mean and Moving Range).

In [5]:
def x_bar_and_mr(orientation, frame, time_period):
    if orientation == 'Front':
        aoa_df = front_aoa_df.copy()
        acs_df = front_acs_df.copy()
    else:
        aoa_df = side_aoa_df.copy()
        acs_df = side_acs_df.copy()
#     aoa_df['Facility'] = 'AOA'
#     acs_df['Facility'] = 'ACS'
#     df1 = pd.concat([aoa_df, acs_df])
            
    aoa_df = aoa_df[aoa_df['Frame'] == frame]
    acs_df = acs_df[acs_df['Frame'] == frame]
    
    if time_period.lower() == 'all':
        scores_aoa = aoa_df.copy()
        scores_acs = acs_df.copy()
    else:
        scores_aoa = aoa_df.resample(time_period).mean().dropna()[['Raw Score']]
        scores_acs = acs_df.resample(time_period).mean().dropna()[['Raw Score']]

    ranges_aoa = scores_aoa.copy()
    ranges_acs = scores_acs.copy()
    
    ranges_aoa = ranges_aoa.shift(1)
    ranges_acs = ranges_acs.shift(1)
    
    ranges_aoa['Prev'] = ranges_aoa['Raw Score']
    ranges_acs['Prev'] = ranges_acs['Raw Score']
    
    ranges_aoa['Range'] = scores_aoa['Raw Score'] - ranges_aoa['Prev']
    ranges_acs['Range'] = scores_acs['Raw Score'] - ranges_acs['Prev']

    # Define plot
    fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 10), sharex=True)
    fig.suptitle(r'$\bar{{{0}}}$ & MR chart ({1}, {2} ms)'.format('X', orientation, frame), fontsize=20)
    # X Bar chart
    sns.lineplot(ax=ax1, y='Raw Score', x=scores_aoa.index, marker='o', data=scores_aoa, color='k')
    sns.lineplot(ax=ax1, y='Raw Score', x=scores_aoa.index, marker='o', data=scores_acs, color='y')
    mean = scores_aoa['Raw Score'].mean()
    cl = 3*scores_aoa['Raw Score'].std()
    ucl = mean + cl
    lcl = mean - cl
    ax1.axhline(scores_aoa['Raw Score'].mean(), color='b')
    ax1.axhline(ucl, color='r', linestyle='dashed')
    ax1.axhline(lcl, color='r', linestyle='dashed')
    ax1.set_ylabel(r'$\bar{{{0}}}$'.format('X'))
    ax1.set_xlabel('')

    # Moving Range chart
    sns.lineplot(ax=ax2, y='Range', x=ranges_aoa.index, marker='o', data=ranges_aoa, color='k')
    sns.lineplot(ax=ax2, y='Range', x=ranges_acs.index, marker='o', data=ranges_acs, color='y')
    mean = ranges_aoa['Range'].mean()
    cl = 3*ranges_aoa['Range'].std()
    ucl = mean + cl
    lcl = mean - cl
    ax2.axhline(ranges_aoa['Range'].mean(), color='b')
    ax2.axhline(ucl, color='r', linestyle='dashed')
    ax2.axhline(lcl, color='r', linestyle='dashed')
    ax2.set_ylabel('MR')
    ax2.set_xlabel('Date')
    plt.savefig('xbarmr.png')
    plt.show()

In [6]:
def comparison_graph(orientation, frame, plot_type, split, show_points):
    front_df = front_aoa_df.copy()
    front_df['AOA'] = front_aoa_df['Raw Score']
    front_df['ACS'] = front_acs_df['Raw Score']
    front_df['orientation'] = 'Front'
    front_df = front_df[front_df['Frame'] == frame]
    front_df_m = pd.melt(front_df[['AOA', 'ACS', 'orientation']], id_vars=['orientation'], var_name='Facility')
    
    side_df = side_aoa_df.copy()
    side_df['AOA'] = side_aoa_df['Raw Score']
    side_df['ACS'] = side_acs_df['Raw Score']
    side_df['orientation'] = 'Side'
    side_df = side_df[side_df['Frame'] == frame]
    side_df_m = pd.melt(side_df[['AOA', 'ACS', 'orientation']], id_vars=['orientation'], var_name='Facility')
    
    this_df = pd.concat([front_df_m, side_df_m])
    
    if orientation.lower() == 'all':
        pass
    else:
        this_df = this_df[this_df['orientation'] == orientation]

    # Plotting section
    if split == True and orientation.lower() != 'all':
        print('Orientation:', orientation)
        print('Error, orientation must be set to All for split plots')
        return
    
    fig, ax = plt.subplots(figsize=(15, 10))
    
    if plot_type == 'violin':
        if split == True:
            sns.violinplot(data=this_df, x='Facility', y='value', hue='orientation', split=True, inner='quart')
            if show_points:
                sns.swarmplot(data=this_df, x='Facility', y='value', color='k', hue='orientation', alpha=0.5, dodge=True)
        else:
            sns.violinplot(data=this_df, x='Facility', y='value', inner='quart')
            if show_points:
                sns.swarmplot(data=this_df, x='Facility', y='value', color='k', alpha=0.5)
    elif plot_type == 'box':
        if split == True:
            sns.boxplot(data=this_df, x='Facility', y='value', hue='orientation')
            if show_points:
                sns.swarmplot(data=this_df, x='Facility', y='value', hue='orientation', color='k', alpha=0.5, dodge=True)
        else:
            sns.boxplot(data=this_df, x='Facility', y='value')
            if show_points:
                sns.swarmplot(data=this_df, x='Facility', y='value', color='k', alpha=0.5)
    
    fig.suptitle(r'Pixel-wise Mean Squared Error at {0} ms'.format(frame), fontsize=20)
    plt.ylabel('Error Score')
    plt.show()
    
# comparison_graph('Front', 10, 'violin', True, True)

This snippet provides a simple user interface to select the frame and time to group the data by. It is defined here, but implemented in section 2.

In [7]:
dialog_xbmr = widgets.Output()
charts_xbmr = widgets.Output()

frames = front_aoa_df.sort_values(by='Frame')['Frame'].unique().tolist()
lbl_frame = widgets.Label('Frame Number (ms)')
drop_down_frame_1 = widgets.Dropdown(options = frames)
lbl_orientation = widgets.Label('Orientation (Front/side)')
drop_down_orientation_1 = widgets.Dropdown(options = ['Front', 'Side'])
lbl_time = widgets.Label('Time Period for Resampling')
drop_down_time = widgets.Dropdown(options = [('All', 'all'), ('Day', 'D'), ('Week', 'W'), ('Month', 'M')])
btn = widgets.Button(description='Refresh XBar and MR chart')
label_frames = widgets.VBox([lbl_frame, drop_down_frame_1, lbl_orientation, drop_down_orientation_1])
label_dropdowns = widgets.VBox([lbl_time, drop_down_time])
labels = widgets.HBox([label_frames, label_dropdowns])

def show_chart(b):
    with charts_xbmr:
        charts_xbmr.clear_output()
        x_bar_and_mr(drop_down_orientation_1.value, drop_down_frame_1.value, drop_down_time.value)
            
with dialog_xbmr:
    display(labels)
    display(btn)
    
    btn.on_click(show_chart)

In [8]:
dialog_comp = widgets.Output()
charts_comp = widgets.Output()

drop_down_frame_2 = widgets.Dropdown(options = frames)
drop_down_orientation_2 = widgets.Dropdown(options = ['All', 'Front', 'Side'])
lbl_plot = widgets.Label('Plot Type')
drop_down_plot = widgets.Dropdown(options = [('Violin', 'violin'), ('Box', 'box')])
# lbl_split = widgets.Label('Split by Orientation')
check_split = widgets.Checkbox(description='Split by Orientation')
# lbl_points = widgets.Label('Show Points')
check_points = widgets.Checkbox(description='Show Points')

btn_comp = widgets.Button(description='Refresh')

labels_1 = widgets.VBox([lbl_frame, drop_down_frame_2, 
                         lbl_orientation, drop_down_orientation_2, 
                         lbl_plot, drop_down_plot])
labels_2 = widgets.VBox([check_split, check_points])
labels = widgets.HBox([labels_1, labels_2])

def show_chart(b):
    with charts_comp:
        charts_comp.clear_output()
        comparison_graph(drop_down_orientation_2.value, 
                         drop_down_frame_2.value, 
                         drop_down_plot.value,
                         check_split.value,
                         check_points.value)
        
with dialog_comp:
    display(labels)
    display(btn_comp)
    
    btn_comp.on_click(show_chart)

<a id='results'></a>
## 2 Results 

### 2.1 $\bar{X}$ and MR Chart

In [9]:
display(dialog_xbmr)
display(charts_xbmr)

Output()

Output()

### 2.2 Comparison Charts

In [10]:
display(dialog_comp)
display(charts_comp)

Output()

Output()